# L4: Support Data Insight Analysis

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

## Loading Tasks and Agents YAML files

In [2]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Using FileReadTool

In [3]:
from crewai_tools import FileReadTool
csv_tool = FileReadTool(file_path='./support_tickets_data.csv')

## Creating Agents, Tasks and Crew

In [4]:
# Creating Agents
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  tools=[csv_tool]
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  tools=[csv_tool]
)

chart_generation_agent = Agent(
  config=agents_config['chart_generation_agent'],
  allow_code_execution=True
)

# Creating Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

chart_generation = Task(
  config=tasks_config['chart_generation'],
  agent=chart_generation_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation, chart_generation]
)


# Creating Crew
support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent,
    chart_generation_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    chart_generation,
    final_report_assembly
  ],
  verbose=True
)


## Testing our Crew

In [7]:
support_report_crew.test(n_iterations=1, eval_llm='gpt-4o')

Invalid type LLM for attribute 'model_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


╭──────────────────────────────────────────────── Test Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  🧪 Starting Crew Test                                                                                          │
│                                                                                                                 │
│  Crew: crew                                                                                                     │
│  ID: 9da37bea-224e-42ac-ac7a-48030598c0ac                                                                       │
│  Iterations: 1                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c5e3a2ef-f57d-4dde-8850-87dde3d6f88f                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Suggestion Engine                                                                                       │
│                                                                                                                 │
│  Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be  │
│  based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use      │
│  historical data such as resolution_time_minutes and                                                            │
│    satisfaction_rating to inform the suggestions.                                                               │
│  - Customer Feedback: Incorporate insights from customer_comments to                                            │
│    customize the suggestions further.                                                                           │
│                                                                                                                 │
│  The goal is to provide clear, actionable steps that the support team can take to resolve each issue            │
│  efficiently and effectively.                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Suggestion Engine                                                                                       │
│                                                                                                                 │
│  Thought: Action: Read a file's content                                                                         │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c5e3a2ef-f57d-4dde-8850-87dde3d6f88f                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Thought: I have gathered the raw data required to compile the report. Now I will analyze it to   │
│  create summary sections including Issue Classification Results, Agent Performance, Customer Satisfaction       │
│  Ratings over time, and Suggested Actions based on the tickets.                                                 │
│                                                                                                                 │
│  ### Final Report                                                                                               │
│                                                                                                                 │
│  ## Issue Classification Results                                                                                │
│                                                                                                                 │
│  | Issue Type          | Frequency | Priority Level                  |                                          │
│  |---------------------|-----------|---------------------------------|                                          │
│  | API Issue           | 8         | High, Critical                  |                                          │
│  | Login Issue         | 7         | High, Critical                  |                                          │
│  | Report Generation    | 7         | Low, High                       |                                         │
│  | Data Import         | 13        | High, Medium                    |                                          │
│  | Feature Request     | 8         | High, Critical                  |                                          │
│  | Billing Issue       | 11        | High, Medium, Critical          |                                          │
│  | UI Bug              | 6         | Medium, Low                     |                                          │
│                                                                                                                 │
│  ### Suggested Actions for Each Issue Type                                                                      │
│  1. **API Issues**: Immediate hotfix implementation and regular maintenance updates, along with continuous      │
│  communication with customers.                                                                                  │
│  2. **Login Issues**: Escalation to higher-level support, increased resource allocation, and timely follow-up.  │
│  3. **Report Generation**: Improved troubleshooting processes, dedicated teams for high-priority tickets, and   │
│  follow-ups with customers.                                                                                     │
│  4. **Data Import Issues**: Detailed investigation into recurring issues, dedicated teams for data import       │
│  handling, and direct customer engagement.                                                                      │
│  5. **Feature Requests**: Regular discussions with customers on implementation timelines and transparency in    │
│  updates.                                                                                                       │
│  6. **Billing Issues**: Enhanced training for agents, 

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

                             Tasks Scores                              
                        (1-10 Higher is better)                        
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━┯━━┓
┃ Tasks/Crew/Agents  │ Run 1 │ Avg. Total │ Agents              │  │  ┃
┠────────────────────┼───────┼────────────┼─────────────────────┼──┼──┨
┃ Task 1             │  8.0  │    8.0     │ - Suggestion Engine │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 2             │  8.0  │    8.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 3             │  2.0  │    2.0     │ - Chart Specialist  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 4             │  8.5  │    8.5     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Crew               │ 6.62  │    6.6     │                     │  │  ┃
┃ Execution Time (s) │  83   │     83     │                     │  │  ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━┷━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┷━━┷━━┛

╭──────────────────────────────────────────────── Test Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Test Execution Completed                                                                                       │
│  Crew: crew                                                                                                     │
│                                                                                                                 │
│  Status: Completed                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Training your crew and agents

In [8]:
support_report_crew.train(n_iterations=1, filename='training.pkl')

╭─────────────────────────────────────────────── Training Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  📋 Crew Training Started                                                                                       │
│  Crew: crew                                                                                                     │
│  Time: 2025-09-28 08:45:54.191766+00:00                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ae26e16c-311b-4ad4-857c-206303109b3d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Suggestion Engine                                                                                       │
│                                                                                                                 │
│  Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be  │
│  based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use      │
│  historical data such as resolution_time_minutes and                                                            │
│    satisfaction_rating to inform the suggestions.                                                               │
│  - Customer Feedback: Incorporate insights from customer_comments to                                            │
│    customize the suggestions further.                                                                           │
│                                                                                                                 │
│  The goal is to provide clear, actionable steps that the support team can take to resolve each issue            │
│  efficiently and effectively.                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Suggestion Engine                                                                                       │
│                                                                                                                 │
│  Thought: Action: Read a file's content                                                                         │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"file_path\": \"./support_tickets_data.csv\", \"start_line\": 1, \"line_count\": null}"                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────── Trace Batch Finalization ────────────────────────────────────────────╮
│ ✅ Trace batch finalized with session ID: 81928028-5a4b-42fd-be4d-1ac3fb4a759f                                  │
│                                                                                                                 │
│ 🔗 View here:                                                                                                   │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/81928028-5a4b-42fd-be4d-1ac3fb4a759f?access_code=TRA │
│ CE-591211e845                                                                                                   │
│ 🔑 Access Code: TRACE-591211e845                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────── 🔍 Execution Trace Generated ──────────────────────────────────────────╮
│                                                                                                                 │
│  🎉 Your First CrewAI Execution Trace is Ready!                                                                 │
│                                                                                                                 │
│  View your execution details here:                                                                              │
│  https://app.crewai.com/crewai_plus/ephemeral_trace_batches/81928028-5a4b-42fd-be4d-1ac3fb4a759f?access_code=T  │
│  RACE-591211e845                                                                                                │
│                                                                                                                 │
│  This trace shows:                                                                                              │
│  • Agent decisions and interactions                                                                             │
│  • Task execution timeline                                                                                      │
│  • Tool usage and results                                                                                       │
│  • LLM calls and responses                                                                                      │
│                                                                                                                 │
│  ✅ Tracing has been enabled for future runs! (CREWAI_TRACING_ENABLED=true added to .env)                       │
│  You can also add tracing=True to your Crew(tracing=True) / Flow(tracing=True) for more control.                │
│                                                                                                                 │
│  📝 Note: This link will expire in 24 hours.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Training Failure ────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ Crew Training Failed                                                                                        │
│  Crew: crew                                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


[2025-09-28 14:47:41][ERROR]: Training failed: Critical training data error: Missing fields (human_feedback) for agent 2ebd98ff-323e-4915-b589-3947e71bcb41 in iteration 0.
This indicates a broken training process. Cannot proceed with evaluation.
Please check your training implementation.


ValueError: Critical training data error: Missing fields (human_feedback) for agent 2ebd98ff-323e-4915-b589-3947e71bcb41 in iteration 0.
This indicates a broken training process. Cannot proceed with evaluation.
Please check your training implementation.

## Comparing new test results

In [9]:
support_report_crew.test(n_iterations=1, eval_llm='gpt-4o')

Invalid type LLM for attribute 'model_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


╭──────────────────────────────────────────────── Test Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  🧪 Starting Crew Test                                                                                          │
│                                                                                                                 │
│  Crew: crew                                                                                                     │
│  ID: 9da37bea-224e-42ac-ac7a-48030598c0ac                                                                       │
│  Iterations: 1                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 54861537-ea52-4194-93b0-66e405a3781f                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Suggestion Engine                                                                                       │
│                                                                                                                 │
│  Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be  │
│  based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use      │
│  historical data such as resolution_time_minutes and                                                            │
│    satisfaction_rating to inform the suggestions.                                                               │
│  - Customer Feedback: Incorporate insights from customer_comments to                                            │
│    customize the suggestions further.                                                                           │
│                                                                                                                 │
│  The goal is to provide clear, actionable steps that the support team can take to resolve each issue            │
│  efficiently and effectively.                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Suggestion Engine                                                                                       │
│                                                                                                                 │
│  Thought: Action: Read a file's content                                                                         │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 54861537-ea52-4194-93b0-66e405a3781f                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### Final Report on Support System Performance                                                   │
│                                                                                                                 │
│  #### Issue Classification Results                                                                              │
│  | Issue Type          | Frequency | Priority Levels                          |                                 │
│  |---------------------|-----------|------------------------------------------|                                 │
│  | API Issue           | 9         | High: 4, Medium: 2, Low: 2, Critical: 1 |                                  │
│  | Login Issue         | 7         | High: 3, Medium: 2, Low: 2, Critical: 2 |                                  │
│  | Report Generation    | 5         | High: 2, Medium: 2, Low: 1              |                                 │
│  | Data Import         | 12        | High: 4, Medium: 4, Low: 4, Critical: 0 |                                  │
│  | Feature Request     | 9         | High: 4, Medium: 3, Low: 1, Critical: 1 |                                  │
│  | Billing Issue       | 11        | High: 4, Medium: 4, Low: 2, Critical: 3 |                                  │
│  | UI Bug              | 7         | High: 2, Medium: 3, Low: 2, Critical: 0 |                                  │
│                                                                                                                 │
│  #### Suggested Actions                                                                                         │
│  - **API Issues**: Implement a more efficient logging system to track future API issues for quicker             │
│  resolution.                                                                                                    │
│  - **Login Issues**: Consider a dedicated team for immediate response to critical login issues and enhance      │
│  training for agents.                                                                                           │
│  - **Report Generation**: Provide additional training for agents on common issues related to report generation  │
│  to improve resolution time and hold review sessions for process improvements.                                  │
│  - **Data Import**: Review workflow and ensure agent training on resolution strategies to decrease resolution   │
│  time.                                                                                                          │
│  - **Feature Request**: Develop tracking mechanisms for feature requests to avoid re-submissions and ensure     │
│  regular updates to customers on request status.                                                                │
│  - **Billing Issues**: Institute a streamlined approach for common problems and conduct training sessions       │
│  focused on efficiency.                                                                                         │
│  - **UI Bugs**: Set up a cross-functional team to address critical UI issues proactively.                       │
│                                                                                                                 │
│  #### Agent Performance                               

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

                             Tasks Scores                              
                        (1-10 Higher is better)                        
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━┯━━┓
┃ Tasks/Crew/Agents  │ Run 1 │ Avg. Total │ Agents              │  │  ┃
┠────────────────────┼───────┼────────────┼─────────────────────┼──┼──┨
┃ Task 1             │  7.0  │    7.0     │ - Suggestion Engine │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 2             │  8.5  │    8.5     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 3             │  8.5  │    8.5     │ - Chart Specialist  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 4             │  8.0  │    8.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Crew               │ 8.00  │    8.0     │                     │  │  ┃
┃ Execution Time (s) │  170  │    170     │                     │  │  ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━┷━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┷━━┷━━┛

╭──────────────────────────────────────────────── Test Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Test Execution Completed                                                                                       │
│  Crew: crew                                                                                                     │
│                                                                                                                 │
│  Status: Completed                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Kicking off Crew

In [10]:
result = support_report_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9da37bea-224e-42ac-ac7a-48030598c0ac                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Suggestion Engine                                                                                       │
│                                                                                                                 │
│  Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be  │
│  based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use      │
│  historical data such as resolution_time_minutes and                                                            │
│    satisfaction_rating to inform the suggestions.                                                               │
│  - Customer Feedback: Incorporate insights from customer_comments to                                            │
│    customize the suggestions further.                                                                           │
│                                                                                                                 │
│  The goal is to provide clear, actionable steps that the support team can take to resolve each issue            │
│  efficiently and effectively.                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Suggestion Engine                                                                                       │
│                                                                                                                 │
│  Thought: Action: Read a file's content                                                                         │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"file_path\": \"./support_tickets_data.csv\", \"start_line\": 1, \"line_count\": null}"                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_t  │
│  ime_minutes,satisfaction_rating,customer_comments,agent_id,resolved                                            │
│  T0001,C0511,API Issue,I'm pleased with how my issue was handled. Thanks!,High,2023-03-25,240,927,4,I'm         │
│  pleased with how my issue was handled. Thanks!,A004,True                                                       │
│  T0002,C0729,Login Issue,Excellent service! The agent went above and beyond.,Low,2023-04-06,223,534,5,The       │
│  problem still persists. Not resolved yet.,A004,False                                                           │
│  T0003,C0439,Report Generation,Resolution was satisfactory but could be                                         │
│  improved.,Low,2023-04-25,214,592,1,"The issue was escalated quickly, which was appreciated.",A004,False        │
│  T0004,C0100,Data Import,Agent was very helpful and polite. Great service!,High,2023-02-20,110,864,2,The issue  │
│  was resolved quickly. Very satisfied!,A003,True                                                                │
│  T0005,C0258,Feature Request,"My issue was handled well, but follow-up could be                                 │
│  better.",Medium,2023-01-18,151,193,2,The problem still persists. Not resolved yet.,A004,False                  │
│  T0006,C0526,Data Import,It took too long to resolve the issue. Not happy.,Low,2023-06-10,134,194,2,This is     │
│  the second time I'm facing this issue. Frustrating!,A002,True                                                  │
│  T0007,C0415,Feature Request,It took too long to resolve the issue. Not happy.,High,2023-01-10,64,232,1,The     │
│  support team was unresponsive at first.,A001,False                                                             │
│  T0008,C0602,Login Issue,Resolution was satisfactory but could be improved.,Critical,2023-02-22,132,578,5,Good  │
│  support but the resolution could have been faster.,A002,True                                                   │
│  T0009,C0934,API Issue,It took too long to resolve the issue. Not happy.,High,2023-06-23,57,46,4,This is the    │
│  second time I'm facing this issue. Frustrating!,A004,False                                                     │
│  T0010,C0410,Feature Request,Good support but the resolution could have been                                    │
│  faster.,Critical,2023-03-28,223,1230,3,This was a critical issue that took too long to resolve.,A002,True      │
│  T0011,C0966,Report Generation,Agent was very helpful and polite. Great                                         │
│  service!,High,2023-03-12,192,1284,2,The issue was ignored for too long. Disapp...                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Generator                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Final Report: Support System Analysis**                                                                      │
│                                                                                                                 │
│  **1. Issue Classification Results**                                                                            │
│                                                                                                                 │
│  | Issue Type         | Frequency | Priority Level |                                                            │
│  |---------------------|-----------|----------------|                                                           │
│  | API Issue           | 8         | High           |                                                           │
│  | Login Issue         | 10        | Medium         |                                                           │
│  | Report Generation    | 6         | Critical       |                                                          │
│  | Data Import         | 10        | Low            |                                                           │
│  | Feature Request     | 9         | Medium         |                                                           │
│  | Billing Issue       | 10        | High           |                                                           │
│  | UI Bug              | 6         | Medium         |                                                           │
│  | **Total**           | **69**    |                |                                                           │
│                                                                                                                 │
│  **2. Agent Performance**                                                                                       │
│                                                                                                                 │
│  | Agent ID | Total Resolutions | Avg. Resolution Time (min) | Avg. Satisfaction Rating |                       │
│  |----------|-------------------|---------------------------|--------------------------|                        │
│  | A001     | 15                | 573.33                    | 3.00                     |                        │
│  | A002     | 12                | 608.92                    | 3.00                     |                        │
│  | A003     | 12                | 580.58                    | 3.25                     |                        │
│  | A004     | 16                | 618.75                    | 3.87                     |                        │
│  | A005     | 12                | 1010.25                   | 4.00                     |                        │
│  | **Total**| **67**            |                           |                          |                        │
│                                                                                                                 │
│  **3. Customer Satisfaction Over Time**                                                                         │
│                                                                                                                 │
│  | Month         | Issues Resolved | Avg. Satisfaction Rating |                                                 │
│  |---------------|------------------|------------------

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 393176e8-b565-4524-83e4-451dbe8b5468                                                                     │
│  Agent: Report Generator                                                                                        │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9da37bea-224e-42ac-ac7a-48030598c0ac                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Final Report: Support System Analysis**                                                        │
│                                                                                                                 │
│  **1. Issue Classification Results**                                                                            │
│                                                                                                                 │
│  | Issue Type         | Frequency | Priority Level |                                                            │
│  |---------------------|-----------|----------------|                                                           │
│  | API Issue           | 8         | High           |                                                           │
│  | Login Issue         | 10        | Medium         |                                                           │
│  | Report Generation    | 6         | Critical       |                                                          │
│  | Data Import         | 10        | Low            |                                                           │
│  | Feature Request     | 9         | Medium         |                                                           │
│  | Billing Issue       | 10        | High           |                                                           │
│  | UI Bug              | 6         | Medium         |                                                           │
│  | **Total**           | **69**    |                |                                                           │
│                                                                                                                 │
│  **2. Agent Performance**                                                                                       │
│                                                                                                                 │
│  | Agent ID | Total Resolutions | Avg. Resolution Time (min) | Avg. Satisfaction Rating |                       │
│  |----------|-------------------|---------------------------|--------------------------|                        │
│  | A001     | 15                | 573.33                    | 3.00                     |                        │
│  | A002     | 12                | 608.92                    | 3.00                     |                        │
│  | A003     | 12                | 580.58                    | 3.25                     |                        │
│  | A004     | 16                | 618.75                    | 3.87                     |                        │
│  | A005     | 12                | 1010.25                   | 4.00                     |                        │
│  | **Total**| **67**            |                           |                          |                        │
│                                                                                                                 │
│  **3. Customer Satisfaction Over Time**                                                                         │
│                                                                                                                 │
│  | Month         | Issues Resolved | Avg. Satisfaction

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

## Result

In [11]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

**Final Report: Support System Analysis**

**1. Issue Classification Results**

| Issue Type         | Frequency | Priority Level | 
|---------------------|-----------|----------------| 
| API Issue           | 8         | High           | 
| Login Issue         | 10        | Medium         | 
| Report Generation    | 6         | Critical       | 
| Data Import         | 10        | Low            | 
| Feature Request     | 9         | Medium         | 
| Billing Issue       | 10        | High           | 
| UI Bug              | 6         | Medium         | 
| **Total**           | **69**    |                | 

**2. Agent Performance**

| Agent ID | Total Resolutions | Avg. Resolution Time (min) | Avg. Satisfaction Rating | 
|----------|-------------------|---------------------------|--------------------------| 
| A001     | 15                | 573.33                    | 3.00                     | 
| A002     | 12                | 608.92                    | 3.00                     | 
| A003     | 12                | 580.58                    | 3.25                     | 
| A004     | 16                | 618.75                    | 3.87                     | 
| A005     | 12                | 1010.25                   | 4.00                     | 
| **Total**| **67**            |                           |                          |

**3. Customer Satisfaction Over Time**

| Month         | Issues Resolved | Avg. Satisfaction Rating |
|---------------|------------------|--------------------------| 
| January       | 12               | 3.12                     | 
| February      | 14               | 3.48                     | 
| March         | 12               | 3.25                     | 
| April         | 15               | 3.75                     | 
| May           | 13               | 3.85                     | 
| June          | 12               | 3.50                     | 
| July          | 1                | 3.00                     | 

**4. Suggested Actions**

- **API Issues**: Maintain high standards of service and follow-up.
- **Login Issues**: Revisit and verify with system logs while providing alternatives.
- **Report Generation**: Enhance communication and periodic updates.
- **Data Import Issues**: Analyze the full resolution process and refine.
- **Feature Requests**: Implement quicker resolution strategies and training.
- **Billing Issues**: Set urgent processes and improve follow-up strategies.
- **UI Bugs**: Increase monitoring and conduct checks for repetitive cases.

**Visual Insights**

- **Issue Distribution Chart**: ![issue_distribution.png](issue_distribution.png)
- **Agent Performance Chart**: ![agent_performance.png](agent_performance.png)
- **Customer Satisfaction Over Time Chart**: ![customer_satisfaction.png](customer_satisfaction.png)
- **Priority Levels Chart**: ![priority_levels.png](priority_levels.png)
- **Average Resolution Times Trend Line**: ![avg_resolution_times.png](avg_resolution_times.png)

This comprehensive report integrates the essential insights from the support data and provides directions for potential improvements in the support system. Stakeholders can use this information to guide decision-making and enhance the quality of service delivery.